# Auto Generated Agent Chat: Using RetrieveChat for Retrieve Augmented Code Generation and Question Answering

RetrieveChat is a convesational framework for retrieve augmented code generation and question answering. In this notebook, we demonstrate how to utilize RetrieveChat to generate code and answer questions based on customized documentations that are not present in the LLM's training dataset. RetrieveChat uses the `RetrieveAssistantAgent` and `RetrieveUserProxyAgent`, which is similar to the usage of `AssistantAgent` and `UserProxyAgent` in other notebooks (e.g., [Automated Task Solving with Code Generation, Execution & Debugging](https://github.com/microsoft/FLAML/blob/main/notebook/autogen_agentchat_auto_feedback_from_code_execution.ipynb)). Essentially,`RetrieveAssistantAgent` and  `RetrieveUserProxyAgent` implements a different auto reply mechanism corresponding to the RetrieveChat prompts.

## Requirements

FLAML requires `Python>=3.8`. To run this notebook example, please install flaml with the [mathchat] option.
```bash
pip install flaml[retrievechat]
```

In [1]:
# %pip install flaml[retrievechat]~=2.0.0rc4

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.


In [2]:
from flaml import oai

config_list = oai.config_list_from_json(
    env_or_file=".config.local",
    file_location=".",
    filter_dict={
        "model": {
            "gpt-4",
            "gpt4",
            "gpt-4-32k",
            "gpt-4-32k-0314",
            "gpt-3.5-turbo",
            "gpt-3.5-turbo-16k",
            "gpt-3.5-turbo-0301",
            "chatgpt-35-turbo-0301",
            "gpt-35-turbo-v0301",
        }
    },
)

assert len(config_list) > 0
print("models to use: ", [config_list[i]["model"] for i in range(len(config_list))])

models to use:  ['gpt-4']


It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the gpt-4 and gpt-3.5-turbo models are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.

You can set the value of config_list in other ways you prefer, e.g., loading from a YAML file.

## Construct agents for RetrieveChat

We start by initialzing the `RetrieveAssistantAgent` and `RetrieveUserProxyAgent`. The system message needs to be set to "You are a helpful assistant." for RetrieveAssistantAgent. The detailed instructions are given in the user message. Later we will use the `RetrieveUserProxyAgent.generate_init_prompt` to combine the instructions and a math problem for an initial prompt to be sent to the LLM assistant.

In [3]:
from flaml.autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from flaml.autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

oai.ChatCompletion.start_logging()

# 1. create an RetrieveAssistantAgent instance named "assistant"
assistant = RetrieveAssistantAgent(
    name="assistant", 
    system_message="You are a helpful assistant.",
    oai_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    },
)

# 2. create the RetrieveUserProxyAgent instance named "ragproxyagent"
# By default, the human_input_mode is "ALWAYS", which means the agent will ask for human input at every step. We set it to "NEVER" here.
# `docs_path` is the path to the docs directory. By default, it is set to "./docs". Here we generated the documentations from FLAML's docstrings.
# Navigate to the website folder and run `pydoc-markdown` and it will generate folder `reference` under `website/docs`.
# `chunk_token_size` is the chunk token size for the retrieve chat. By default, it is set to `max_tokens * 0.6`, here we set it to 2000.
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    retrieve_config={"docs_path": "../website/docs/reference", "chunk_token_size": 2000},
)

### Example 1

Use RetrieveChat to help generate sample code and automatically run the code and fix errors if there is any.

Problem: Which API should I use if I want to use FLAML for a classification task and I want to train the model in 30 seconds. Use spark to parallel the training. Force cancel jobs if time limit is reached.

In [4]:
import os

os.environ["ALL_PROXY"] = ""


In [5]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# given a problem, we use the ragproxyagent to generate a prompt to be sent to the assistant as the initial message.
# the assistant receives the message and generates a response. The response will be sent back to the ragproxyagent for processing.
# The conversation continues until the termination condition is met, in RetrieveChat, the termination condition when no human-in-loop is no code block detected.
# With human-in-loop, the conversation will continue until the user says "exit".
code_problem = "Which API should I use if I want to use FLAML for a classification task and I want to train the model in 30 seconds. Use spark to parallel the training. Force cancel jobs if time limit is reached."
ragproxyagent.initiate_chat(assistant, problem=code_problem, search_string="spark")  # search_string is used as an extra filter for the embeddings search, in this case, we only want to search documents that contain "spark".

doc_ids:  [['doc_35', 'doc_40', 'doc_14', 'doc_51', 'doc_21', 'doc_15', 'doc_52', 'doc_41', 'doc_44', 'doc_36', 'doc_13', 'doc_8', 'doc_38', 'doc_45']]
Adding doc_id doc_35 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a QA task.
Step 2, you generate code or answer the question based on the intent.
You should leverage the context provided by the user as much as possible. If you think the context is not enough, you
can reply exactly "UPDATE CONTEXT" to ask the user to provide more contexts.
For code generation, you must obey the following rules:
You MUST NOT install any packages because all the packages needed are already installed.
The code will be executed in IPython, you must follow 

NameError: name 'X_train' is not defined

ragproxyagent (to assistant):

exitcode: 1 (execution failed)
Code output: 
None
name 'X_train' is not defined

--------------------------------------------------------------------------------
[flaml.autogen.oai.completion: 07-31 21:11:54] {212} INFO - retrying in 10 seconds...
Traceback (most recent call last):
  File "/home/lijiang1/code/FLAML1/flaml/autogen/oai/completion.py", line 204, in _get_response
    response = openai_completion.create(**config)
  File "/home/lijiang1/anaconda3/envs/flaml-oss/lib/python3.8/site-packages/openai/api_resources/chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "/home/lijiang1/anaconda3/envs/flaml-oss/lib/python3.8/site-packages/openai/api_resources/abstract/engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "/home/lijiang1/anaconda3/envs/flaml-oss/lib/python3.8/site-packages/openai/api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_r

INFO:flaml.autogen.oai.completion:retrying in 10 seconds...
Traceback (most recent call last):
  File "/home/lijiang1/code/FLAML1/flaml/autogen/oai/completion.py", line 204, in _get_response
    response = openai_completion.create(**config)
  File "/home/lijiang1/anaconda3/envs/flaml-oss/lib/python3.8/site-packages/openai/api_resources/chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "/home/lijiang1/anaconda3/envs/flaml-oss/lib/python3.8/site-packages/openai/api_resources/abstract/engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "/home/lijiang1/anaconda3/envs/flaml-oss/lib/python3.8/site-packages/openai/api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "/home/lijiang1/anaconda3/envs/flaml-oss/lib/python3.8/site-packages/openai/api_requestor.py", line 700, in _interpret_response
    self._interpret_response_line(
  File "/home/lijiang1/an

assistant (to ragproxyagent):

Apologies for the confusion. The provided code snippet assumes that you have already loaded your dataset and defined the `X_train` and `y_train` variables. Since I do not have access to your dataset, please make sure you load your dataset and split it into training and testing sets before executing the provided code. Here's a revised example using a sample dataset from the `sklearn` library:

```python
from flaml import AutoML
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Load a sample dataset
data = load_iris()
X = data.data
y = data.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate AutoML
automl = AutoML()

# Set AutoML configuration
automl_settings = {
    "time_budget": 30,  # In seconds
    "problem_type": "classification",
    "use_spark": True,
    "n_concurrent_trials": 1,  # or any desi

...

[flaml.tune.tune: 07-31 21:13:02] {729} INFO - Number of trials: 1/1000000, 1 RUNNING, 0 TERMINATED


[flaml.tune.tune: 07-31 21:13:08] {729} INFO - Number of trials: 1/1000000, 1 RUNNING, 0 TERMINATED


[flaml.tune.tune: 07-31 21:13:15] {729} INFO - Number of trials: 1/1000000, 1 RUNNING, 0 TERMINATED


Time exceeded, canceled jobs                                        (0 + 1) / 1]



assistant (to ragproxyagent):

TERMINATE

--------------------------------------------------------------------------------
TERMINATE

--------------------------------------------------------------------------------


### Example 2

Use RetrieveChat to answer a question that is not related to code generation.

Problem: Who is the author of FLAML?

In [6]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()
qa_problem = "Who is the author of FLAML?"
ragproxyagent.initiate_chat(assistant, problem=qa_problem)

doc_ids:  [['doc_58', 'doc_51', 'doc_35', 'doc_3', 'doc_22', 'doc_40', 'doc_14', 'doc_13', 'doc_59', 'doc_52', 'doc_1', 'doc_6', 'doc_28', 'doc_56', 'doc_29', 'doc_2', 'doc_55', 'doc_44', 'doc_19', 'doc_32']]
Adding doc_id doc_58 to context.
ragproxyagent (to assistant):



You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a QA task.
Step 2, you generate code or answer the question based on the intent.
You should leverage the context provided by the user as much as possible. If you think the context is not enough, you
can reply exactly "UPDATE CONTEXT" to ask the user to provide more contexts.
For code generation, you must obey the following rules:
You MUST NOT install any packages because all the packages needed are already installed.
The code will be executed in IPython, you must follow the formats below to write your code:
```python
# your code
```

User's question is: Who is the author of FLAML?

Context is: ---
sidebar_label: estimator
title: default.estimator
---

#### flamlize\_estimator

```py

### Example 3

Use RetrieveChat to help generate sample code and ask for human-in-loop feedbacks.

Problem: how to build a time series forecasting model for stock price using FLAML?

In [8]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# set `human_input_mode` to be `ALWAYS`, so the agent will ask for human input at every step.
ragproxyagent.human_input_mode = "ALWAYS"
code_problem = "how to build a time series forecasting model for stock price using FLAML?"
ragproxyagent.initiate_chat(assistant, problem=code_problem)

doc_ids:  [['doc_37', 'doc_35', 'doc_46', 'doc_40', 'doc_49', 'doc_51', 'doc_58', 'doc_48', 'doc_14', 'doc_39', 'doc_47', 'doc_41', 'doc_13', 'doc_60', 'doc_52', 'doc_59', 'doc_43', 'doc_10', 'doc_34', 'doc_33']]
Adding doc_id doc_37 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a QA task.
Step 2, you generate code or answer the question based on the intent.
You should leverage the context provided by the user as much as possible. If you think the context is not enough, you
can reply exactly "UPDATE CONTEXT" to ask the user to provide more contexts.
For code generation, you must obey the following rules:
You MUST NOT install any packages because all the packages needed are already inst

### Example 4

Use RetrieveChat to answer a question and ask for human-in-loop feedbacks.

Problem: Is there a function named `tune_automl` in FLAML?

In [9]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# set `human_input_mode` to be `ALWAYS`, so the agent will ask for human input at every step.
ragproxyagent.human_input_mode = "ALWAYS"
qa_problem = "Is there a function named `tune_automl` in FLAML?"
ragproxyagent.initiate_chat(assistant, problem=qa_problem)

doc_ids:  [['doc_35', 'doc_14', 'doc_40', 'doc_13', 'doc_51', 'doc_58', 'doc_20', 'doc_26', 'doc_34', 'doc_22', 'doc_11', 'doc_59', 'doc_3', 'doc_56', 'doc_47', 'doc_53', 'doc_19', 'doc_28', 'doc_32', 'doc_46']]
Adding doc_id doc_35 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a QA task.
Step 2, you generate code or answer the question based on the intent.
You should leverage the context provided by the user as much as possible. If you think the context is not enough, you
can reply exactly "UPDATE CONTEXT" to ask the user to provide more contexts.
For code generation, you must obey the following rules:
You MUST NOT install any packages because all the packages needed are already insta